In [ ]:
import numpy as np

In [1]:
def Rx(theta):
    return np.matrix([[ 1, 0           , 0           ],
                   [ 0, np.cos(theta),-np.sin(theta)],
                   [ 0, np.sin(theta), np.cos(theta)]])
  
def Ry(theta):
    return np.matrix([[ np.cos(theta), 0, np.sin(theta)],
                   [ 0           , 1, 0           ],
                   [-np.sin(theta), 0, np.cos(theta)]])
  
def Rz(theta):
    return np.matrix([[ np.cos(theta), -np.sin(theta), 0 ],
                   [ np.sin(theta), np.cos(theta) , 0 ],
                   [ 0           , 0            , 1 ]])
def R2d(theta):
    return np.matrix([[ np.cos(theta), -np.sin(theta) ],
                   [ np.sin(theta), np.cos(theta) ]])

def Sph2Cart(r,th,ph, deg: bool = True):
    if (deg==True):
        if th.any()<-0.5 :
            raise Exception('Latitude must be in range [0,180]. Try 90-Lat.')
        th = np.radians(th) #Latitude
        ph = np.radians(ph) #Longitud 
    else:
        if th.any()<-0.5 :
            raise Exception('Latitude must be in range [0, Pi]. Try Pi/2-Lat.')
    x = r*np.sin(th)*np.cos(ph)
    y = r*np.sin(th)*np.sin(ph)
    z = r*np.cos(th)
    return np.array([x, y, z ])

def Sph2CartE(r,th,ph,er,et,ep, deg: bool = True):
    if (deg==True):
        if th.any()<-0.5 :
            raise Exception('Latitude must be in range [0,180]. Try 90-Lat.')
        th = np.radians(th) #Latitude
        ph = np.radians(ph) #Longitud 
        et = np.radians(et) #ErrLat
        ep = np.radians(ep) #ErrLon 
    else:
        if th.any()<-0.5 :
            raise Exception('Latitude must be in range [0, Pi]. Try Pi/2-Lat.')
    x  = r*np.sin(th)*np.cos(ph)
    y  = r*np.sin(th)*np.sin(ph)
    z  = r*np.cos(th)

    # Compute partial derivatives
    partial_x_r = np.sin(th) * np.cos(ph)
    partial_x_theta = r * np.cos(th) * np.cos(ph)
    partial_x_phi = -r * np.sin(th) * np.sin(ph)
    
    partial_y_r = np.sin(th) * np.sin(ph)
    partial_y_theta = r * np.cos(th) * np.sin(ph)
    partial_y_phi = r * np.sin(th) * np.cos(ph)
    
    partial_z_r = np.cos(th)
    partial_z_theta = -r * np.sin(th)
    #artial_z_phi = 0
    
    # Calculate errors
    ex = np.sqrt(
        (partial_x_r * er) ** 2 +
        (partial_x_theta * et) ** 2 +
        (partial_x_phi * ep) ** 2
    )
    
    ey = np.sqrt(
        (partial_y_r * er) ** 2 +
        (partial_y_theta * et) ** 2 +
        (partial_y_phi * ep) ** 2
    )
    
    ez = np.sqrt(
        (partial_z_r * er) ** 2 +
        (partial_z_theta * et) ** 2
    )
    

    return np.array([x, y, z, ex, ey, ez])

def Cart2Sph(x,y,z, deg: bool = True):
    XY2 = x**2 + y**2
    r = np.sqrt(XY2 + z**2)             # r
    th = np.arctan2(z,np.sqrt(XY2))     # theta
    ph = np.arctan2(y,x)                # phi
    if (deg==True):
        return np.array([r, np.degrees(th), np.mod(np.degrees(ph),360)])
    else:
        return np.array([r,th,ph])

def Cart2SphE(x,y,z,ex,ey,ez, deg: bool = True):
    XY2 = x**2 + y**2
    r = np.sqrt(XY2 + z**2)             # r
    th = np.arctan2(z,np.sqrt(XY2))     # theta
    ph = np.arctan2(y,x)                # phi



    # Compute partial derivatives
    partial_r_x = x / r
    partial_r_y = y / r
    partial_r_z = z / r
    
    denom_theta = r**2 * np.sqrt(1 - (z / r)**2)
    partial_theta_x = -x * z / denom_theta
    partial_theta_y = -y * z / denom_theta
    partial_theta_z = r / (r**2 * np.sqrt(1 - (z / r)**2))
    
    partial_phi_x = -y / XY2
    partial_phi_y = x / XY2
    #partial_phi_z = 0
    
    # Calculate errors
    er = np.sqrt(
        (partial_r_x * ex) ** 2 +
        (partial_r_y * ey) ** 2 +
        (partial_r_z * ez) ** 2
    )
    
    et = np.sqrt(
        (partial_theta_x * ex) ** 2 +
        (partial_theta_y * ey) ** 2 +
        (partial_theta_z * ez) ** 2
    )
    
    ep = np.sqrt(
        (partial_phi_x * ex) ** 2 +
        (partial_phi_y * ey) ** 2 +
        (partial_phi_z * ez) ** 2
    )
    
    if (deg==True):
        return np.array([r, np.degrees(th), np.mod(np.degrees(ph),360),er,np.degrees(et),np.degrees(ep)])
    else:
        return np.array([r,th,ph,er,et,ep])
    
def vecSph2Cart(vr,vt,vp,th,ph, deg: bool = True):
    if (deg==True):
        th = np.radians(th) #Latitude
        ph = np.radians(ph) #Longitud
    vx = vr*np.sin(th)* np.cos(ph) + vt*np.cos(th)* np.cos(ph) - vp*np.sin(ph)
    vy = vr*np.sin(th)* np.sin(ph) + vt*np.cos(th)* np.sin(ph) + vp*np.cos(ph)
    vz = vr*np.cos(th)             - vt*np.sin(th)
    return np.array([vx, vy, vz])
           

def GrCirc(ph, th0, ph0):
    return np.arctan2(np.tan(th0)*np.sin(ph0-ph),np.sin(ph0))

def RotCen(hf,k,lat,lon,tilt, ne_t, deg: bool = True):
    # Central height and radio
    h = hf/(1.+k)
    # Rotate circle to lat, lon, tilt and ne-tilt
    if (deg==True):
        lat = np.radians(lat) 
        lon = np.radians(lon) 
        tilt= np.radians(tilt)
        ne_t= np.radians(ne_t)     
    R1 = Ry(-ne_t) #Rotation counter-clockwise
    xc2,yc2,zc2 = np.array(R1*np.array([[h-1],[0],[0]]))
    xc2 = xc2+1
    R2 = Rz(lon)*Ry(-lat)*Rx(tilt) #Rotation counter-clockwise
    return R2 * [xc2,yc2,zc2]

def RotCirc(hf,k,lat,lon,tilt, ne_t, num=50,cen:bool = True, deg: bool = True):
    # Central height and radio
    h = hf/(1.+k)
    a = hf-h
    # circle projected in lat, lon (0,0)
    psi = np.linspace(0, 2*np.pi,num)
    x1 = h + a* np.cos(psi)
    y1 = np.zeros(num)
    z1 = a* np.sin(psi)
    # Rotate circle to lat, lon, tilt and ne-tilt
    if (deg==True):
        lat = np.radians(lat) 
        lon = np.radians(lon) 
        tilt= np.radians(tilt)
        ne_t= np.radians(ne_t)
        
    R1 = Ry(-ne_t) #Rotation counter-clockwise
    xr2,yr2,zr2 = np.array(R1 * [x1-1,y1,z1]) 
    xr2 = xr2+1
    xc2,yc2,zc2 = np.array(R1*np.array([[h-1],[0],[0]]))
    xc2 = xc2+1
    R2 = Rz(lon)*Ry(-lat)*Rx(tilt) #Rotation counter-clockwise
    if (cen==False):
        return np.array(R2 * [xr2,yr2,zr2])
    else:
        return np.array(R2 * [xr2,yr2,zr2]), np.array(R2 * [xc2,yc2,zc2])

from numpy import pi, sin, cos, tan, arcsin, sqrt
from numpy.linalg import norm

## From Github: johan12345/gcs_python

def skeleton(alpha, distjunc, straight_vertices, front_vertices, k):
    """
    Compute the axis of the GCS CME model. Based on IDL version shellskeleton.pro
    https://hesperia.gsfc.nasa.gov/ssw/stereo/secchi/idl/scraytrace/shellskeleton.pro
    Parameters
    ----------
    alpha: width of CME (half angle)
    distjunc: CME height (in length units, e.g. solar radii)
    straight_vertices: number of vertices along straight edges
    front_vertices: number of vertices along front
    k: GCS ratio
    Returns
    -------
    """
    # compute entire loop lenght
    loop_length = distjunc * (1. + (alpha +pi / 2) * tan(alpha))
    # copute circular part half length
    circle_length = distjunc * np.tan(alpha) * (2. * alpha +pi) / 2

    gamma = arcsin(k)

    # calculate the points of the straight line part
    pRstart = np.array([1, sin(alpha), cos(alpha)])  # start on the limb
    pLstart = np.array([1, -sin(alpha), cos(alpha)])
    pslR = np.outer(np.linspace(1, distjunc, straight_vertices), np.array([0, sin(alpha), cos(alpha)]))
    pslL = np.outer(np.linspace(1, distjunc, straight_vertices), np.array([0, -sin(alpha), cos(alpha)]))
    rsl = tan(gamma) * norm(pslR, axis=1)
    casl = np.full(straight_vertices, -alpha)

    # calculate the points of the circular part
    beta = np.linspace(-alpha, pi / 2, front_vertices)
    hf = distjunc
    h = hf / cos(alpha)
    rho = hf * tan(alpha)

    X0 = (rho + h * k ** 2 * sin(beta)) / (1 - k ** 2)
    rc = sqrt((h ** 2 * k ** 2 - rho ** 2) / (1 - k ** 2) + X0 ** 2)
    cac = beta

    pcR = np.array([np.zeros(beta.shape), X0 * cos(beta), h + X0 * sin(beta)]).T
    pcL = np.array([np.zeros(beta.shape), -X0 * cos(beta), h + X0 * sin(beta)]).T

    r = np.concatenate((rsl, rc[1:], np.flipud(rc)[1:], np.flipud(rsl)[1:]))
    ca = np.concatenate((casl, cac[1:], pi-np.flipud(cac)[1:], pi-np.flipud(casl)[1:]))
    p = np.concatenate((pslR, pcR[1:], np.flipud(pcL)[1:], np.flipud(pslL)[1:]))

    return p, r, ca


def gcs_mesh(alpha, hf, k, straight_vertices=10, front_vertices=10, circle_vertices=10):
    """
    Calculate GCS model mesh. Based on IDL version cmecloud.pro.
    https://hesperia.gsfc.nasa.gov/ssw/stereo/secchi/idl/scraytrace/cmecloud.pro
    Parameters
    ----------
    alpha: width of CME (half angle, in radians)
    height: CME height (in length units, e.g. solar radii)
    straight_vertices: number of vertices along straight edges
    front_vertices: number of vertices along front
    circle_vertices: number of vertices along each circle
    k: GCS ratio
    Returns
    -------
    GCS mesh (in length units, e.g. solar radii) in the following coordinate system (Heliographic Stonyhurst):
    - Z axis: Sun-Earth line projected onto Sun's equatorial plane
    - Y axis: Sun's north pole
    - Origin: center of the Sun
    """
    # calculate position

    # calculate height of skeleton depending on height of leading edge
    distjunc = hf *(1-k)*cos(alpha)/(1.+sin(alpha))
    p, r, ca = skeleton(alpha, distjunc, straight_vertices, front_vertices, k)

    theta = np.linspace(0, 2 * pi, circle_vertices)
    pspace = np.arange(0, (front_vertices + straight_vertices) * 2 - 3)
    u, v = np.meshgrid(theta, pspace)
    x_u = u.shape[0]
    y_u = u.shape[1]
    u, v = u.flatten(), v.flatten()

    mesh = r[v, np.newaxis] * np.array([cos(u), sin(u) * cos(ca[v]), sin(u) * sin(ca[v])]).T + p[v]

    # mesh = np.concatenate([
    #    r * np.array([costheta, sintheta * cos(ca), sintheta * sin(ca)]).T + np.array([x, y, 0])
    #    for (x, y), r, ca in zip(p, r, ca)
    # ])
    mesh_s = mesh.reshape(x_u,y_u,3)
    return mesh, mesh_s

def gcs_mesh_rotated(alpha, hf, k, lat, lon, tilt,ne_t,
                     straight_vertices=int(10),front_vertices=int(10),circle_vertices=int(10), deg: bool = True):
    """
    Calculate GCS model mesh and rotate it into the correct orientation.
    Convenience function that combines gcs_mesh and rotate_mesh.
    Parameters
    ----------
    alpha: width of CME (half angle, in radians)
    height: CME height (in length units, e.g. solar radii)
    straight_vertices: number of vertices along straight edges
    front_vertices: number of vertices along front
    circle_vertices: number of vertices along each circle
    k: GCS ratio
    lat: CME latitude in radians
    lon: CME longitude in radians (0 = Earth-directed)
    tilt: CME tilt angle in radians
    Returns
    -------
    rotated GCS mesh
    """
    # Rotate circle to lat, lon, tilt and ne-tilt
    if (deg==True):
        lat = np.radians(lat)-pi/2  #Skeleton defined in z=1
        lon = np.radians(lon) 
        tilt= np.radians(tilt)
        ne_t= np.radians(ne_t)
        alpha = np.radians(alpha)
        
    mesh, mesh_s = gcs_mesh(alpha, hf,k, straight_vertices, front_vertices, circle_vertices)
    R1 = Ry(-ne_t) #Rotation counter-clockwise
    xr1,yr1,zr1 = np.array(R1 * [mesh[:,0],mesh[:,1],mesh[:,2]-1]) 
    zr1 = zr1+1
    
    R2 = Rz(lon)*Ry(-lat)*Rz(tilt) #Rotation counter-clockwise
    
    mesh_r = np.array(R2 * [xr1,yr1,zr1])
    return mesh_r.reshape(3,mesh_s.shape[0],mesh_s.shape[1])